In [1]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast
from fuzzywuzzy import process
from nltk.corpus import words

## Initial Experiments

In [ ]:
total = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\total_summary_stats_updated.csv")

In [ ]:
patient_id = total["patient_id"].to_list()
patient_id = patient_id[:10]

In [ ]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches.csv")

In [ ]:
# ONLY RUN ONCE #

# vertifying that all match records can be found
for i in range(len(patient_id)):
    specific_matches = matches[matches["PatientID"] == patient_id[i]]
    if (len(specific_matches) <= 0):
        print("NOT FOUND: ", patient_id[i])
    #specific_matches.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_{}.csv".format(i+1))

In [ ]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_1.csv")

In [ ]:
unique_ids = matches["PatientID"].unique().tolist()
print(len(unique_ids))

In [ ]:
# interesting match cases
# patient_2 in specific study row 6, cream that has mci inside it

In [ ]:
print(subset_loc_clone)

## Final Code

In [68]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\apoe_4_matches.csv")

In [69]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\Sentence-Analysis\EDA\Getting_Data\keywords.csv")

In [70]:
regex

,REGEX
0,\sdementia
1,\scognition
2,\smemory
3,\sCDR
4,\sMMSE
5,\sMOCA
6,\sAlzheimer
7,\scognitive\s*impairment
8,\sMCI
9,\scerebellar


In [71]:
len(p1)

265127

In [72]:
locations = []
for r in tqdm(p1['regex_sent']):
    try:
        locations.append([match.span() \
                  for match in re.finditer(re.compile("|".join(regex["REGEX"]), 
                                                  re.IGNORECASE),r)])
    except:
        locations.append([])
        
p1["pruned_regex_location"] = locations

100%|███████████████████████████████████████████████████████████████████████| 265127/265127 [00:08<00:00, 30867.20it/s]


In [73]:
len(p1)

265127

In [74]:
p1 = p1[p1["pruned_regex_location"].str.len() != 0]
len(p1)

205006

In [75]:
total_merged = []

for index, row in p1.iterrows():
    curr = str(row["regex_location"])
    curr = curr[1:len(curr) - 1]
    curr = list((eval(curr)))
    #print("CURR: ", len(curr))
    # print(curr)
    merged = []

    start = 0
    end = 0
    length = 0
    
    if (len(curr) == 2 and isinstance(curr[1], int)):
        start = curr[0]
        end = curr[1]
        #print(start, end)
        length = len(row["NoteTXT"])
        if (start < 100):
            start = 100
        elif (end > length - 100):
            end = length - 100
        curr = [(start - 100, end + 100)]
        
    else:
        loc_list = []
        for j in range(len(curr)):
            start = curr[j][0]
            end = curr[j][1]
            loc_list.append((start, end))
        length = len(row["NoteTXT"])
        for k in range(len(loc_list)):
            start = loc_list[k][0]
            end = loc_list[k][1]
            if (start < 100):
                start = 100
            elif (end > length - 100):
                end = length - 100
            loc_list[k] = (start - 100, end + 100)
        curr = loc_list
        
    for interval in curr:
        if not merged:
            merged.append(interval)
            continue

        prevInterval = merged.pop()
        if prevInterval[0] <= interval[0] <= prevInterval[1]:
            startTime = prevInterval[0]
            endTime = prevInterval[1] if prevInterval[1] > interval[1] else interval[1]
            merged.append((startTime, endTime))
        else:
            merged.append(prevInterval)
            merged.append(interval)
            
    total_merged.append(merged)
p1["merged_row_location"] = total_merged

In [76]:
p1

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count,pruned_regex_location,merged_row_location
0,0,104241704,11454931,Z8126942,3.110872e+09,2016-04-13 00:00:00.0000000,Office Visit,976521500,Progress Notes,4,9.324995e+08,1.0,----------------------------------------------...,"[(2884, 2890), (3964, 3967)]","""rash, pruritus, changing moles Neuro: No head...","[""memory""]",1,"[(97, 104)]","[(2784, 2990), (3864, 4067)]"
11,11,104241704,11454931,Z8126942,3.216313e+09,2018-09-25 00:00:00.0000000,Office Visit,2149500933,Progress Notes,4,2.119050e+09,1.0,----------------------------------------------...,"[(831, 840)]","""hernia. This patient also has a history of at...","[""Alzheimer""]",1,"[(94, 104)]","[(731, 940)]"
12,12,104241704,11454931,Z8126942,3.254832e+09,2019-07-24 00:00:00.0000000,Office Visit,2616753951,Progress Notes,4,2.589719e+09,1.0,----------------------------------------------...,"[(7345, 7353), (10684, 10693)]","""heart failure and cardiac disease in his moth...","[""dementia""]",1,"[(99, 108)]","[(7245, 7453), (10584, 10793)]"
14,14,104241704,11454931,Z8126942,3.254832e+09,2019-07-24 00:00:00.0000000,Office Visit,2616760232,Letter,4,2.589725e+09,1.0,----------------------------------------------...,"[(7843, 7851), (11182, 11191)]","""heart failure and cardiac disease in his moth...","[""dementia""]",1,"[(99, 108)]","[(7743, 7951), (11082, 11291)]"
21,21,104241732,479313,Z8127097,3.227834e+09,2018-12-13 00:00:00.0000000,Office Visit,2253093803,Progress Notes,4,2.218845e+09,1.0,----------------------------------------------...,"[(4264, 4272), (4378, 4386)]","""images were reviewed with the patient NM Rena...","[""cortical""]",1,"[(96, 105)]","[(4164, 4486)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265122,265136,106692571,2871964,Z9527072,3.285416e+09,2020-02-03 00:00:00.0000000,Office Visit,3632696457,Progress Notes,4,3.612970e+09,1.0,----------------------------------------------...,"[(1349, 1358), (1375, 1381)]","""Depression Screening: Depression Screening 2/...","[""Cognition"", ""memory""]",2,"[(66, 76), (92, 99)]","[(1249, 1481)]"
265123,265137,106692571,2871964,Z9527072,3.285416e+09,2020-02-03 00:00:00.0000000,Office Visit,3642289343,Progress Notes,4,3.635128e+09,1.0,----------------------------------------------...,"[(5502, 5511)]","""normal coordination and gait. Psych: Alert an...","[""cognition""]",1,"[(97, 107)]","[(5402, 5611)]"
265124,265138,106692571,2871964,Z9527072,3.344076e+09,2021-02-17 00:00:00.0000000,Office Visit,5323325643,Progress Notes,4,5.333696e+09,1.0,----------------------------------------------...,"[(3883, 3892)]","""normal coordination and gait. Psych: Alert an...","[""cognition""]",1,"[(97, 107)]","[(3783, 3992)]"
265125,265139,106692571,2871964,Z9527072,3.344076e+09,2021-02-17 00:00:00.0000000,Office Visit,5339569775,Progress Notes,4,5.333713e+09,1.0,----------------------------------------------...,"[(1335, 1344), (1361, 1367)]","""opioid medication(s). Depression Screening: D...","[""Cognition"", ""memory""]",2,"[(89, 99), (115, 122)]","[(1235, 1467)]"


In [77]:
total_merged_note_txt = []
for index, row in p1.iterrows():
    curr = row["merged_row_location"]
    merged_note_txt = ""
    
    for i in range(len(curr)):
        start = curr[i][0]
        end = curr[i][1]
        
        merged_note_txt += " ------- " + row["NoteTXT"][start:end]
    total_merged_note_txt.append(merged_note_txt)
    
p1["merged_regex_sent"] = total_merged_note_txt

In [78]:
len(p1)

205006

In [79]:
p1['merged_regex_sent'] = p1['merged_regex_sent'].str.split(' ',1).str[1].str.rpartition(' ')[0]

In [80]:
#p1['NoteTXT'] = p1['NoteTXT'].astype(str)
p1 = p1.drop_duplicates(subset='merged_regex_sent', keep="first")
p1 = p1.replace(r'^\s*$', np.NaN, regex=True)
p1.reset_index(drop = True)

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count,pruned_regex_location,merged_row_location,merged_regex_sent
0,0,104241704,11454931,Z8126942,3.110872e+09,2016-04-13 00:00:00.0000000,Office Visit,976521500,Progress Notes,4,9.324995e+08,1.0,----------------------------------------------...,"[(2884, 2890), (3964, 3967)]","""rash, pruritus, changing moles Neuro: No head...","[""memory""]",1,"[(97, 104)]","[(2784, 2990), (3864, 4067)]","------- No rash, pruritus, changing moles Neur..."
1,11,104241704,11454931,Z8126942,3.216313e+09,2018-09-25 00:00:00.0000000,Office Visit,2149500933,Progress Notes,4,2.119050e+09,1.0,----------------------------------------------...,"[(831, 840)]","""hernia. This patient also has a history of at...","[""Alzheimer""]",1,"[(94, 104)]","[(731, 940)]",------- ntral hernia. This patient also has a ...
2,12,104241704,11454931,Z8126942,3.254832e+09,2019-07-24 00:00:00.0000000,Office Visit,2616753951,Progress Notes,4,2.589719e+09,1.0,----------------------------------------------...,"[(7345, 7353), (10684, 10693)]","""heart failure and cardiac disease in his moth...","[""dementia""]",1,"[(99, 108)]","[(7245, 7453), (10584, 10793)]",------- heart failure and cardiac disease in ...
3,21,104241732,479313,Z8127097,3.227834e+09,2018-12-13 00:00:00.0000000,Office Visit,2253093803,Progress Notes,4,2.218845e+09,1.0,----------------------------------------------...,"[(4264, 4272), (4378, 4386)]","""images were reviewed with the patient NM Rena...","[""cortical""]",1,"[(96, 105)]","[(4164, 4486)]",------- the images were reviewed with the pati...
4,25,104241732,479313,Z8127097,3.236542e+09,2019-03-07 00:00:00.0000000,Office Visit,2359387299,Progress Notes,4,2.330320e+09,1.0,----------------------------------------------...,"[(4436, 4444), (4550, 4558)]","""stent projects in satisfactory location. NM R...","[""cortical""]",1,"[(99, 108)]","[(4336, 4658)]",------- stent projects in satisfactory locati...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60475,265132,106692571,2871964,Z9527072,3.174241e+09,2018-01-08 00:00:00.0000000,Office Visit,1815696776,Progress Notes,4,1.768529e+09,1.0,----------------------------------------------...,"[(2334, 2343), (2484, 2490)]","""revising it. Advance Care/End of Life Plannin...","[""Cognition""]",1,"[(98, 108)]","[(2234, 2590)]",------- n revising it. Advance Care/End of Lif...
60476,265134,106692571,2871964,Z9527072,3.231995e+09,2019-02-01 00:00:00.0000000,Office Visit,2315556631,Progress Notes,4,2.293248e+09,1.0,----------------------------------------------...,"[(4226, 4235)]","""normal coordination and gait. Psych: Alert an...","[""cognition""]",1,"[(97, 107)]","[(4126, 4335)]",------- normal coordination and gait. [Psych]...
60477,265135,106692571,2871964,Z9527072,3.285416e+09,2020-02-03 00:00:00.0000000,Office Visit,3632696457,Progress Notes,4,3.612970e+09,1.0,----------------------------------------------...,"[(1349, 1358), (1375, 1381)]","""opioid medication(s). Depression Screening: D...","[""Cognition"", ""memory""]",2,"[(88, 98), (114, 121)]","[(1249, 1481)]",------- rescription opioid medication(s). Depr...
60478,265137,106692571,2871964,Z9527072,3.285416e+09,2020-02-03 00:00:00.0000000,Office Visit,3642289343,Progress Notes,4,3.635128e+09,1.0,----------------------------------------------...,"[(5502, 5511)]","""normal coordination and gait. Psych: Alert an...","[""cognition""]",1,"[(97, 107)]","[(5402, 5611)]",------- normal coordination and gait. [Psych]...


In [81]:
p1.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\apoe_4_matches_updated_pruned.csv", index = False)

## Getting #  of Unique Encounters per Patient

In [3]:
## getting number of unique encounters per patient
df = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_matches_updated.csv")

In [ ]:
df = df.drop_duplicates(subset = 'PatientEncounterID', keep = "first")

In [ ]:
summary_stats_1 = df.groupby(['EMPI'])["PatientEncounterID"].count().reset_index()
summary_stats_1

In [ ]:
summary_stats_1.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_summary_stats_updated.csv")

In [ ]:
s1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_summary_stats_updated.csv")
s2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\2\apoe_2_summary_stats_updated.csv")
s3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\3\apoe_3_summary_stats_updated.csv")
s4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\apoe_4_summary_stats_updated.csv")
s5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_summary_stats_updated.csv")

In [ ]:
len(s1)

In [ ]:
len(s2)

In [ ]:
len(s3)

In [ ]:
len(s4)

In [ ]:
len(s5)

In [ ]:
frames = [s1, s2, s3, s4, s5]
total = pd.concat(frames)

In [ ]:
total.reset_index(drop = True)

In [ ]:
total.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\total_summary_stats_updated_7_5_2021.csv")

In [ ]:
subset = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches_updated.csv")

In [ ]:
subset = subset.iloc[0:5000]
subset

In [ ]:
subset.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches_updated_snippet.csv")

## Cleaning up Similar Encounters on Patient Level

In [82]:
df = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\4\apoe_4_matches_updated_pruned.csv")

In [83]:
df.head()

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count,pruned_regex_location,merged_row_location,merged_regex_sent
0,0,104241704,11454931,Z8126942,3.110872e+09,2016-04-13 00:00:00.0000000,Office Visit,976521500,Progress Notes,4,9.324995e+08,1.0,----------------------------------------------...,"[(2884, 2890), (3964, 3967)]","""rash, pruritus, changing moles Neuro: No head...","[""memory""]",1,"[(97, 104)]","[(2784, 2990), (3864, 4067)]","------- No rash, pruritus, changing moles Neur..."
1,11,104241704,11454931,Z8126942,3.216313e+09,2018-09-25 00:00:00.0000000,Office Visit,2149500933,Progress Notes,4,2.119050e+09,1.0,----------------------------------------------...,"[(831, 840)]","""hernia. This patient also has a history of at...","[""Alzheimer""]",1,"[(94, 104)]","[(731, 940)]",------- ntral hernia. This patient also has a ...
2,12,104241704,11454931,Z8126942,3.254832e+09,2019-07-24 00:00:00.0000000,Office Visit,2616753951,Progress Notes,4,2.589719e+09,1.0,----------------------------------------------...,"[(7345, 7353), (10684, 10693)]","""heart failure and cardiac disease in his moth...","[""dementia""]",1,"[(99, 108)]","[(7245, 7453), (10584, 10793)]",------- heart failure and cardiac disease in ...
3,21,104241732,479313,Z8127097,3.227834e+09,2018-12-13 00:00:00.0000000,Office Visit,2253093803,Progress Notes,4,2.218845e+09,1.0,----------------------------------------------...,"[(4264, 4272), (4378, 4386)]","""images were reviewed with the patient NM Rena...","[""cortical""]",1,"[(96, 105)]","[(4164, 4486)]",------- the images were reviewed with the pati...
4,25,104241732,479313,Z8127097,3.236542e+09,2019-03-07 00:00:00.0000000,Office Visit,2359387299,Progress Notes,4,2.330320e+09,1.0,----------------------------------------------...,"[(4436, 4444), (4550, 4558)]","""stent projects in satisfactory location. NM R...","[""cortical""]",1,"[(99, 108)]","[(4336, 4658)]",------- stent projects in satisfactory locati...


In [84]:
# total_merged_sequences = []

# for i in tqdm(unique_empi):
#     merged_sequences = df[df["EMPI"] == i]["merged_regex_sent"].values
#     #print("MERGED: ", merged_sequences, "\n")

#     counter = 0
#     for j in merged_sequences:
# #         choices = []
        
# #         for k in merged_sequences:
# #             if (k != j):
# #                 choices.append(k)
# #         choices = merged_sequences[0:counter]
# #         choices += merged_sequences[counter+1:]
#         #print("\n------------------")
#         choices = list(merged_sequences[0:counter]) + list(merged_sequences[counter+1:])
#         result = get_match(j, choices)
#         score = result[0][1]
#         txt = result[0][0]
        
#         if (score >= 90):
#             # 90 is duplicate string threshold
#             idx = df[df["merged_regex_sent"] == str(txt)].index[0]
#             df.drop([idx])
        
#         counter += 1

In [85]:
len(df)

60480

In [86]:
df["PatientEncounterID"].nunique()

38950

In [87]:
df["PatientEncounterID"].isna().sum()

0

In [88]:
df = df.drop_duplicates(subset='merged_regex_sent', keep="first")

In [89]:
len(df)

60480

## Data Visualization

In [ ]:
x = ((7509, 7518), (7735, 7741), (7745, 7754), (7858, 7862))
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(str(y))
z = len(p1[(p1["regex_location"] == str(y)) & (p1["PatientEncounterID"] == int(encounters[21]))]['NoteTXT'].values[1])
z

In [ ]:
x = [(181, 390),
  (479, 688),
  (2048, 2269),
  (6184, 6390),
  (6584, 6821),
  (7450, 7659),
  (7716, 7932),
  (9864, 10072),
  (11192, 11648),
  (12118, 12627)]
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(y)

In [ ]:
len(p1[(p1["regex_location"] == "[" +str(subset_loc[1]) + "]") & (p1["PatientEncounterID"] == int(encounters[1]))]['NoteTXT'].values[0])